<a href="https://colab.research.google.com/github/AmandinChyba/Project1-Fields-2022/blob/main/Laplace_On_Disc/LaplaceOnDisc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [194]:
from jax import random
from jax import lax

import jax.numpy as jnp
import numpy as np

import time
import matplotlib.pyplot as plt

from functools import partial

In [195]:
def poisson_kernel(r, theta):
  return ((1 - jnp.power(r, 2)) / 
          (1 - 2 * r * jnp.cos(theta) + jnp.power(r, 2)))

In [466]:
def metroStep(cur, prop, kernel, r, theta):
  k1 = kernel(r, theta-prop[0])
  k2 = kernel(r, theta-cur)
  A = jnp.minimum(1, jnp.divide(k1,k2)) - prop[1]
  Y = jnp.ceil(A)
  out = jnp.multiply(cur, 1-Y) + jnp.multiply(prop[0], Y)
  return out, (out, jnp.array([0.0]))


def solve_at_point(g, k, r, theta, batches, t, key):
  keys = random.split(key, 2)

  proposals = random.uniform(keys[0], shape=(t, batches)) * 2 * jnp.pi
  U = random.uniform(keys[1], shape=(t-1, batches))

  metroFunc = partial(metroStep, kernel=poisson_kernel, r=r, theta=theta)
  final, result = lax.scan(metroFunc, proposals[0], (proposals[1:], U))
  return jnp.mean(g(result[0]))

In [489]:
%%time
batches = 50
t = 10000
r = 0.5
theta = jnp.pi / 2
key = random.PRNGKey(int(time.time()))
ans = solve_at_point(jnp.sin, poisson_kernel, r, theta, batches, t, key)
print(ans)

0.5046108
CPU times: user 201 ms, sys: 7.89 ms, total: 209 ms
Wall time: 198 ms
